In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.info()

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [26]:
#drop id columns, ones that are not necessary

data = [train, test]

df = pd.concat(data, ignore_index = True)

#drop id cols nd unnecessary
cols = ['ID','Customer_ID','Month','Name','SSN',]

df = df.drop(cols, axis = 1)
df

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,23,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,-500,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,23,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,4975,Architect,20002.88,1929.906667,10,8,29,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,25,18.31,12.0,_,3571.7,34.780553,NaN,Yes,60.964772,146.48632477751087,Low_spent_Small_value_payments,275.53956951573343,NaN
149996,25,Mechanic,39628.99,NaN,4,6,7,2_,"Auto Loan, and Student Loan",20,NaN,11.5,7.0,Good,502.38,27.758522,31 Years and 11 Months,NM,35.104023,181.44299902757518,Low_spent_Small_value_payments,409.39456169535066,NaN
149997,25,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",23,5,13.5,7.0,Good,502.38,36.858542,32 Years and 0 Months,No,35.104023,__10000__,Low_spent_Large_value_payments,349.7263321025098,NaN
149998,25,Mechanic,39628.99,NaN,4,6,7,2_,"Auto Loan, and Student Loan",21,6_,11.5,7.0,Good,502.38,39.139840,32 Years and 1 Months,No,35.104023,97.59857973344877,High_spent_Small_value_payments,463.23898098947717,NaN


In [27]:
#drop na values from columns

df = df.dropna()
df

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
6,23,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,8_,11.27,4.0,Good,809.98,22.537593,22 Years and 7 Months,No,49.574949,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good
8,28_,_______,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,3,4,5.42,2.0,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
9,28,Teacher,34847.84,3037.986667,2,4,6,1,Credit-Builder Loan,7,1,7.42,2.0,Good,605.03,38.550848,26 Years and 8 Months,No,18.816215,40.39123782853101,High_spent_Large_value_payments,484.5912142650067,Good
10,28,Teacher,34847.84_,3037.986667,2,1385,6,1,Credit-Builder Loan,3,-1,5.42,2.0,_,605.03,33.224951,26 Years and 9 Months,No,18.816215,58.51597569589465,High_spent_Large_value_payments,466.46647639764313,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,25,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",20,6,9.5,3.0,_,502.38,39.323569,31 Years and 5 Months,No,35.104023,140.58140274528395,High_spent_Medium_value_payments,410.256158,Poor
99995,25,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",23,7,11.5,3.0,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.866228,Poor
99996,25,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",18,7,11.5,3.0,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.65161,Poor
99997,25,Mechanic,39628.99,3359.415833,4,6,5729,2,"Auto Loan, and Student Loan",27,6,11.5,3.0,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.809083,Poor


In [30]:
for col in df.columns:
    print(col + ": ")
    print("")
    print(df[col].value_counts())
    print("")

Age: 

38       1728
28       1706
34       1656
26       1639
36       1638
         ... 
7549        1
4583_       1
3988_       1
5741        1
1342        1
Name: Age, Length: 1148, dtype: int64

Occupation: 

_______          4143
Lawyer           3972
Mechanic         3786
Doctor           3742
Architect        3702
Entrepreneur     3701
Teacher          3685
Media_Manager    3676
Scientist        3660
Engineer         3657
Accountant       3657
Developer        3650
Writer           3485
Journalist       3479
Manager          3469
Musician         3469
Name: Occupation, dtype: int64

Annual_Income: 

20867.67      12
9141.63       11
40341.16      11
17816.75      10
95596.35      10
              ..
28578.92_      1
15157.25_      1
130832.37_     1
29101.2_       1
39628.99_      1
Name: Annual_Income, Length: 15084, dtype: int64

Monthly_Inhand_Salary: 

6358.956667    13
6639.560000    11
6769.130000    11
6082.187500    11
1315.560833     9
               ..
2756.856952    

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58933 entries, 0 to 99999
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       58933 non-null  int64  
 1   Occupation                58933 non-null  object 
 2   Annual_Income             58933 non-null  object 
 3   Monthly_Inhand_Salary     58933 non-null  float64
 4   Num_Bank_Accounts         58933 non-null  int64  
 5   Num_Credit_Card           58933 non-null  int64  
 6   Interest_Rate             58933 non-null  int64  
 7   Num_of_Loan               58933 non-null  object 
 8   Type_of_Loan              58933 non-null  object 
 9   Delay_from_due_date       58933 non-null  int64  
 10  Num_of_Delayed_Payment    58933 non-null  object 
 11  Changed_Credit_Limit      58933 non-null  object 
 12  Num_Credit_Inquiries      58933 non-null  float64
 13  Credit_Mix                58933 non-null  object 
 14  Outsta

In [37]:
#from the above we can see that we need to change some noise within the columns (values that dont makes sense)

#remove all incorrect characters
sym = "\\`*_{}[]()>#@+!$:;"

int_cols = ['Age','Monthly_Inhand_Salary','Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Delay_from_due_date','Num_of_Delayed_Payment','Outstanding_Debt',
           'Total_EMI_per_month','Monthly_Balance','Annual_Income']

str_cols = ['Occupation','Credit_History_Age','Payment_of_Min_Amount','Credit_Score']
#fix int cols
for i in int_cols:
  for c in sym:
    df[i] = df[i].astype(str).str.replace(c,'')
    
#fix str cols
for i in str_cols:
  for c in sym:
    df[i] = df[i].replace(c,'')
    
    

/tmp/ipykernel_123/3853637528.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[i] = df[i].astype(str).str.replace(c,'')


In [39]:
df.head(20)

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,1824.8433333333328,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.57494921489417,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
6,23,Scientist,19114.12,1824.8433333333328,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,8,11.27,4.0,Good,809.98,22.537593,22 Years and 7 Months,No,49.57494921489417,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good
8,28,_______,34847.84,3037.986666666666,2,4,6,1,Credit-Builder Loan,3,4,5.42,2.0,Good,605.03,24.464031,26 Years and 7 Months,No,18.816214573128885,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
9,28,Teacher,34847.84,3037.986666666666,2,4,6,1,Credit-Builder Loan,7,1,7.42,2.0,Good,605.03,38.550848,26 Years and 8 Months,No,18.816214573128885,40.39123782853101,High_spent_Large_value_payments,484.5912142650067,Good
10,28,Teacher,34847.84,3037.986666666666,2,1385,6,1,Credit-Builder Loan,3,-1,5.42,2.0,_,605.03,33.224951,26 Years and 9 Months,No,18.816214573128885,58.51597569589465,High_spent_Large_value_payments,466.46647639764313,Standard
12,28,Teacher,34847.84,3037.986666666666,2,4,6,1,Credit-Builder Loan,3,1,6.42,2.0,Good,605.03,34.977895,26 Years and 11 Months,No,18.816214573128885,130.11542024292334,Low_spent_Small_value_payments,444.8670318506144,Good
13,28,Teacher,34847.84,3037.986666666666,2,4,6,1,Credit-Builder Loan,3,0,5.42,2.0,Good,605.03,33.381010,27 Years and 0 Months,No,18.816214573128885,43.477190144355745,High_spent_Large_value_payments,481.505261949182,Good
15,28,Teacher,34847.84,3037.986666666666,2,4,6,1,Credit-Builder Loan,3,4,5.42,2.0,Good,605.03,32.933856,27 Years and 2 Months,No,18.816214573128885,218.90434353388733,Low_spent_Small_value_payments,356.07810855965045,Good
16,34,_______,143162.64,12187.22,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",5,8,7.1,3.0,Good,1303.01,28.616735,17 Years and 9 Months,No,246.9923194537421,168.413702679309,!@9#%8,1043.3159778669492,Good
17,34,Engineer,143162.64,12187.22,1,5,8,3,"Auto Loan, Auto Loan, and Not Specified",13,6,7.1,3.0,Good,1303.01,41.702573,17 Years and 10 Months,No,246.9923194537421,232.86038375993544,High_spent_Small_value_payments,998.8692967863226,Good


In [41]:
for col in df.columns:
    print(col + ": ")
    print("")
    print(df[col].value_counts())
    print("")

Age: 

38      1825
28      1790
34      1738
36      1735
26      1728
        ... 
7549       1
4583       1
3988       1
5741       1
1342       1
Name: Age, Length: 1097, dtype: int64

Occupation: 

_______          4143
Lawyer           3972
Mechanic         3786
Doctor           3742
Architect        3702
Entrepreneur     3701
Teacher          3685
Media_Manager    3676
Scientist        3660
Engineer         3657
Accountant       3657
Developer        3650
Writer           3485
Journalist       3479
Manager          3469
Musician         3469
Name: Occupation, dtype: int64

Annual_Income: 

40341.16      13
9141.63       12
20867.67      12
22434.16      11
17816.75      11
              ..
15012905.0     1
11883149.0     1
17914656.0     1
17474415.0     1
20501034.0     1
Name: Annual_Income, Length: 11653, dtype: int64

Monthly_Inhand_Salary: 

6358.956666666666     13
6639.56               11
6769.13               11
6082.1875             11
1315.5608333333332     9
         

In [45]:
#continue editing
df = df[df['Occupation'].str.contains('_______') == False]
df = df[df['Payment_Behaviour'].str.contains('!@9#%8') == False]
df = df[df['Amount_invested_monthly'].str.contains('__10000__') == False]
df = df[df['Monthly_Balance'].str.contains('-333333333333333333333333333') == False]

#credit_mix is copy of credit score but with false '_' values, type of loan has too many levels to categorize into levels
cols_to_drop = ['Credit_Mix', 'Type_of_Loan']
df = df.drop(cols_to_drop, axis = 1)

for col in df.columns:
    print(col + ": ")
    print("")
    print(df[col].value_counts())
    print("")

Age: 

38      1491
28      1455
26      1447
34      1439
36      1417
        ... 
3779       1
6611       1
6227       1
6111       1
1342       1
Name: Age, Length: 901, dtype: int64

Occupation: 

Lawyer           3509
Mechanic         3341
Doctor           3313
Entrepreneur     3300
Teacher          3271
Architect        3271
Accountant       3249
Media_Manager    3231
Scientist        3225
Developer        3218
Engineer         3202
Journalist       3094
Manager          3080
Musician         3078
Writer           3044
Name: Occupation, dtype: int64

Annual_Income: 

22434.16      11
40341.16      10
17816.75       9
64249.44       8
17055.61       8
              ..
2146819.0      1
17706148.0     1
37070.0        1
2617379.0      1
20104809.0     1
Name: Annual_Income, Length: 11524, dtype: int64

Monthly_Inhand_Salary: 

6358.956666666666     11
6082.1875              9
4387.2725              9
1908.874166666667      8
2844.9266666666667     8
                      ..
6251.12

In [46]:
df = df[df['Changed_Credit_Limit'].str.contains('_') == False]


In [47]:
#transform varaibles to be interpretable numerically

df['Credit_History_Age'] = df['Credit_History_Age'].astype(str).str.replace(' Years and ','.')
df['Credit_History_Age'] = df['Credit_History_Age'].astype(str).str.replace('Months','')

#low # indicates low spending, high # indicates high spending
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('Low_spent_Small_value_payments','1')
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('Low_spent_Medium_value_payments','2')
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('Low_spent_Large_value_payments','3')
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('High_spent_Small_value_payments','4')
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('High_spent_Medium_value_payments','5')
df['Payment_Behaviour'] = df['Payment_Behaviour'].astype(str).str.replace('High_spent_Large_value_payments','6')

#here poor values indicate smaller # and good values indicate higher #'s
df['Credit_Score'] = df['Credit_Score'].str.replace('Good', '3', n=-1)
df['Credit_Score'] = df['Credit_Score'].str.replace('Standard', '2', n=-1)
df['Credit_Score'] = df['Credit_Score'].str.replace('Poor', '1', n=-1)
df['Credit_Score'] = df[['Credit_Score']].apply(pd.to_numeric)

df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].str.replace('NM', '0')
df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].str.replace('Yes', '1')
df['Payment_of_Min_Amount'] = df['Payment_of_Min_Amount'].str.replace('No', '2')
df['Payment_of_Min_Amount'] = df[['Payment_of_Min_Amount']].apply(pd.to_numeric)

for col in df.columns:
    print(col + ": ")
    print("")
    print(df[col].value_counts())
    print("")


Age: 

38      1460
28      1429
26      1427
34      1398
36      1392
        ... 
7038       1
7490       1
4975       1
5478       1
1342       1
Name: Age, Length: 882, dtype: int64

Occupation: 

Lawyer           3428
Mechanic         3271
Doctor           3243
Entrepreneur     3220
Architect        3215
Teacher          3209
Accountant       3178
Media_Manager    3173
Scientist        3163
Developer        3153
Engineer         3125
Musician         3032
Journalist       3023
Manager          3013
Writer           2975
Name: Occupation, dtype: int64

Annual_Income: 

22434.16     11
40341.16     10
17816.75      9
62756.2       8
74394.2       8
             ..
17018.45      1
77703.16      1
108312.64     1
17290.42      1
18463.93      1
Name: Annual_Income, Length: 11506, dtype: int64

Monthly_Inhand_Salary: 

6358.956666666666     11
4387.2725              9
6082.1875              9
2637.1116666666667     8
1181.2299999999998     8
                      ..
1352.6575         